In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, fnmatch, shutil

from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict
from PIL import Image, ImageEnhance
from datetime import datetime


In [15]:
def read_to_dataframes(path, frames, x=640, y=480):
    """
    Read to dataframe with specified columns and rows and path
    Parameters:
        path(string)               :   filepath and filename of txt file containing IR data
        x(int)                     :   amount of pixels along x-axis.
        y(int)                     :   amount of pixels along y-axis.
    Returns:
        Data_df(Pandas.DataFrame)  :       Contains float values of temperature measurement for IR cameras
    """
# data is 640x480 in size
    
    album = list()
    
    f = open(path, 'r')
    #contain timestamp
    for k in range(0,frames,1):
        f.readline()
        #Initialise dictionary 
        temp_dct ={str(i): [] for i in range(0,x, 1)}
        #Seperate values and stores in a dictionary
        for j in range(0,y,1):
            string = f.readline()
            str_list = string.split('\t')
            str_list=str_list[0:-1]

            {temp_dct[str(i)].append(float(str_list[i])) for i in range(0,len(str_list), 1)}

        data_df = pd.DataFrame.from_dict(temp_dct)
        album.append(data_df)
    return album

def format_data(data_df, x0=238,x1=396,y0=13,y1=467):
    """
    Only fetch data from specified pixels
    Parameters:
        data_df(DataFrame)  : data_frame containing complete data
        x0(int)             : x start position
        x1(int)             : x end position
        y0(int)             : y start position
        y1(int)             : y end position
    Return:
        raw_arr(array[,])  : formated data
    """
    x0, x1 = str(x0), str(x1)
    data_df = data_df.loc[y0:y1, x0:x1]
    raw_arr = data_df.to_numpy()
    return raw_arr

def normalize_arr(raw_arr):
    """
    Normalize a numpy array based on max and min temp 1800 and 800 respectivly
    Parameters:
        raw_arr(numpy.array)       : A 2D numpy array containing one frame from camera
    Returns:
        raw_arr(numpy.array)       : Normalized array
    """
    low_val_flags = raw_arr < 800
    raw_arr = (raw_arr - 800)/(1800-800)
    raw_arr[low_val_flags] = 0
    raw_arr
    return raw_arr

def sharpen_img(raw_arr, factor = 5):
    """
    Sharpens the image using pillow library
    Parameters:
        raw_arr(numpy.array)        : Normalized array of a frame
        factor(int)                 : Sharpening factor
    Returns:
        im(Pillow.Image)            : A sharpened image based on raw_arr
    """
    im = Image.fromarray(raw_arr*255)
    im = im.convert('RGB')
    enhancer = ImageEnhance.Sharpness(im)

    im = enhancer.enhance(factor)
    return im

def crop_im(im):
    """
    Crops the image to focus on taphole
    Parameters:
        im(Pillow.Image)             : sharpened image
    Returns:
        temp_im(Pillow.Image)        : Image of taphole
    """
    temp_im = im
        
    flamming_area = (0,0,150,180)
    temp_im = temp_im.crop(flamming_area)
    return temp_im

def find_all(path, pattern='*.txt'):
    """
    Will search for any file with .txt extension in path
    Parameters:
        path(string)              : Path to folder with data
        pattern(string)           : Pattern to look for
    Returns:
        result(List(string))      : list containing all paths with .txt extension
    """
    result = []

    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append((root, name))
    return result

# method
def pre_process(path, n_frames = 350, crop = True, save = True):
    """
    Will pre-process and image process raw data in .txt format
    Parameters:
        path(string)               : Path to folder containing raw data
        n_frames(int)              : Number of frames to extract
        crop(bool)                 : True if image will be cropped
        save(bool)                 : True if image will be saved
    """
    img_lst = []
    #Will read a .txt file from path into a list of dataframes, then move the file to a new location
    df = read_camera_data(path, n_frames)
    i = 0
    #for each element in df [list of dataframes]
    for frame in df:
        #Format data and output to a numpy matrix
        raw_arr = format_data(frame)
        #Normalize the data
        raw_arr = normalize_arr(raw_arr)
        #load matrix to an image, then sharpen it
        im = sharpen_img(raw_arr)
        # optional, will crop images for flaming detection
        if crop:
            im = crop_im(im)
        # will append all images to a list, then return it. optional it can save images.
        img_lst.append(im)

        im.save(f'project work/test/processed/test_new{i}.png')
        i+=1
        
def read_camera_data(path, n_frames):
    """
    Read camera data
    Parameters:
        path(string)           : Path to raw data folder
        n_frames(int)          : number of frames to extract
    Returns:
        df(Pandas.DataFrame)   : Dataframe containing raw data.
    """
    path_read = f'{path}/raw'
    path_oldraw = f'{path}/old_raw/'
    
    path_read = find_all(path_read)[0]
    file_name = path_read[1]
    path_read = os.path.join(path_read[0],path_read[1])

    df = read_to_dataframes(path_read, n_frames)
    #shutil.move(path_read, f'{path_oldraw}/{file_name}')
    
    return df

In [16]:
path = 'project work/test'
im_lst = pre_process(path, 100, crop=True)


In [35]:
path = "project work/test/raw"
temp = find_all(path)
temp

2021_09_21_12_57_17_089.txt
test.txt


['project work/test/raw\\name2021_09_21_12_57_17_089.txt',
 'project work/test/raw\\nametest.txt']

In [19]:
shutil.move(r"project work\test\raw\test.txt",r"project work\test\old_raw\test.txt")

'project work\\test\\old_raw\\test.txt'